In [37]:
%load_ext autoreload
%autoreload 2

In [38]:
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from dotenv import load_dotenv
from utils import load_img_path_from_dir, img_to_array
from data_transformation import data_transformation

In [39]:
mopdel_1 = True
# alternativa de modelos : >>>> VGG16, restnet, google vision
def model_2_init():
    input_shape=(255,255,3)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32,kernel_size=(5,5),input_shape=input_shape,padding='same',activation='relu'))
    model.add(tf.keras.layers.Conv2D(64,kernel_size=(5,5),padding='same',activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(4,4)))
    model.add(tf.keras.layers.Conv2D(128,kernel_size=(4,4),padding='same',activation='relu'))
    model.add(tf.keras.layers.Conv2D(160,kernel_size=(4,4),padding='same',activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3)))
    model.add(tf.keras.layers.Conv2D(256,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(64,activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(30,activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(2,activation='softmax'))

    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[tf.keras.metrics.Recall()])
    return model


In [40]:
model = model_2_init()

In [41]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 255, 255, 32)      2432      
                                                                 
 conv2d_16 (Conv2D)          (None, 255, 255, 64)      51264     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 63, 63, 64)       0         
 2D)                                                             
                                                                 
 conv2d_17 (Conv2D)          (None, 63, 63, 128)       131200    
                                                                 
 conv2d_18 (Conv2D)          (None, 63, 63, 160)       327840    
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 21, 21, 160)      0         
 2D)                                                  

In [42]:
dir_train = os.environ.get('TRAIN_PATH')
dir_test = os.environ.get('TEST_PATH')
dir_val = os.environ.get('VAL_PATH')

In [43]:
dataset_train = image_dataset_from_directory(
  dir_train,
  labels = "inferred",
  label_mode = "categorical",
  seed=123,
  image_size=(255,255),
  batch_size=8)

Found 130 files belonging to 2 classes.


In [44]:
dataset_val = image_dataset_from_directory(
    dir_val,
    labels = "inferred",
    label_mode = "categorical",
    seed=123,
    image_size=(255,255),
    batch_size=8
)

Found 318 files belonging to 2 classes.


In [45]:
dataset_test = image_dataset_from_directory(
    dir_test,
    labels = 'inferred',
    label_mode = 'categorical',
    seed=123,
    image_size=(255,255),
    batch_size=8,
)

Found 132 files belonging to 3 classes.


In [46]:
# dataset_train = datagen.flow(dataset_train)

# Apply data augmentation and prefetch the data
# dataset_train = dataset_train.map(lambda x, y: (datagen.random_transform(x), y)).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [47]:
#es_recall = EarlyStoppingByRecall(monitor='val_recall', value=0.90, verbose=1)
es = EarlyStopping(patience=1,restore_best_weights=True)  # Early Stopping normal para la pérdida
model = model_2_init()
history = model.fit(dataset_train,validation_data=dataset_val,epochs=20,callbacks=[es])

Epoch 1/20
17/17 [==============================] - 20s 1s/step - loss: 14.9813 - recall_4: 0.6308 - val_loss: 0.4178 - val_recall_4: 0.9057
Epoch 2/20
17/17 [==============================] - 20s 1s/step - loss: 0.6189 - recall_4: 0.7615 - val_loss: 0.4051 - val_recall_4: 0.9057
Epoch 3/20
17/17 [==============================] - 20s 1s/step - loss: 0.6162 - recall_4: 0.7923 - val_loss: 0.4241 - val_recall_4: 0.9057


In [48]:
# load the images from tests/0 and predict
img_tests_0 = load_img_path_from_dir(os.environ.get('TEST_PATH')+ '/0')
predictions_O = []
for img in img_tests_0:
    img = img_to_array(img)
    predictions_O.append(model.predict(img))

ValueError: in user code:

    File "/Users/puchi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "/Users/puchi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/puchi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/puchi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 1983, in predict_step
        return self(x, training=False)
    File "/Users/puchi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) fro

In [36]:
# load the images from tests/1 and predict
img_tests_1 = load_img_path_from_dir(os.environ.get('TEST_PATH')+ '/1')
predictions_1 = []
for img in img_tests_1:
    img = img_to_array(img)
    img_transformation = data_transformation(img, os.environ.get('TEST_PATH')+ '/t', 1)
    predictions_1.append(model.predict(img))

ValueError: in user code:

    File "/Users/puchi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "/Users/puchi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/puchi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/puchi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 1983, in predict_step
        return self(x, training=False)
    File "/Users/puchi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) fro

In [ ]:
# Obtener la categoria con más probabilidad
for res in predictions_O:
    dog_id = np.argmax(res.max())
    print(dog_id)

In [ ]:
for res in predictions_1:
    dog_id = np.argmax(res.max())
    print(dog_id)

In [ ]:
len(predictions_1)

In [ ]:
len(predictions_O)